In [1]:
%env CUDA_VISIBLE_DEVICES=0

device='cuda'

env: CUDA_VISIBLE_DEVICES=0


In [2]:
if False:
    !pip install mtcnn
    !pip install timm==0.4.5
    !pip install tensorflow
    !pip install numba
    #!pip install keras==2.4.3
    #!pip install -q torch==1.7.1 torchvision
    !pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from copy import deepcopy
from tqdm import tqdm

from mtcnn import MTCNN

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model, model_from_json
from tensorflow.keras.optimizers import Adam, SGD

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from torchvision import datasets, transforms

print(f"Torch: {torch.__version__}")

2024-08-11 08:57:52.342203: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 08:57:52.380901: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 08:57:52.380927: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 08:57:52.380955: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 08:57:52.388268: I tensorflow/core/platform/cpu_feature_g

Torch: 2.1.0+cu121


In [4]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense, Activation, Concatenate, Reshape
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax

In [5]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Reserved:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA A100-PCIE-40GB
Memory Usage:
Allocated: 0.0 GB
Reserved:    0.0 GB


In [6]:
N_CLASSES = 2

In [7]:
USERS_DATASET = "/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/"
FEATURES_DIR = USERS_DATASET + "features_with_balanced/"

In [8]:
DATA_DIR='/home/HDD6TB/datasets/emotions/EmotiW/engagement/'
ext = "png"

# Helpers

In [9]:
import pickle

def save_features(file_name, features):
    if os.path.isfile(file_name):
        print("Error! Cannot save features because file already exists")
        return
    with open(file_name, 'wb') as f:
        pickle.dump(features, f) # , protocol=pickle.HIGHEST_PROTOCOL)
    
def load_features(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

def save_weights(model, file_name):
    if os.path.isfile(file_name):
        print("Error! Cannot save features because file already exists")
        return
    model.save_weights(file_name)

def load_weights(model, file_name):
    model.load_weights(file_name)

In [10]:
test_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

def extract_features(vid2frame):
    vid2features = {}
    #import re
    for j, k in enumerate(vid2frame.keys()):
        print('[{}/{}]'.format(j+1, len(vid2frame)), end="\r")
        #actor_name = re.split(" |_", k)[0]
        imgs = []
        sequence = []
        for i, vid in enumerate(vid2frame[k]):
            img_file = vid
            if torch_model:
                image = Image.open(img_file)
                img_tensor = test_transforms(image)
                imgs.append(img_tensor)
            else:
                image = cv2.imread(img_file)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                imgs.append(image)
            
            if len(imgs) >= BATCH_SIZE:
                if torch_model:
                    features = cnn_model(torch.stack(imgs, dim=0).to(device))
                    features = features.data.cpu().numpy()
                else:
                    features = cnn_model.predict(np.array(imgs), verbose=0)
                if len(sequence) == 0:
                    sequence = features
                else:
                    sequence = np.concatenate((sequence, features),axis=0)
                imgs = []  
        if len(imgs) > 0:        
            if torch_model:
                features = cnn_model(torch.stack(imgs, dim=0).to(device))
                features = features.data.cpu().numpy()
            else:
                features = cnn_model.predict(np.array(imgs), verbose=0)
            if len(sequence) == 0:
                sequence = features
            else:
                sequence = np.concatenate((sequence, features),axis=0)
        
        vid2features[k] = sequence
    return vid2features

def extract_or_read_features(pickle, vid2frame):
    if os.path.isfile(pickle):
        file2features = load_features(pickle)
    else:
        file2features = extract_features(vid2frame)
        save_features(pickle, file2features)
    return file2features

In [11]:
INPUT_SIZE = (224, 224)
model_urls = {
    #'affectnet_7_vggface2_rexnet150.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/affectnet_7_vggface2_rexnet150.pt',
    # 'enet_b0_7.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b0_7.pt',
    'enet_b0_8_best_afew.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b0_8_best_afew.pt',
    # 'enet_b0_8_best_vgaf.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b0_8_best_vgaf.pt',
    # 'enet_b2_7.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b2_7.pt',
    # 'enet_b2_8.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b2_8.pt',
    'mobilenet_7.h5': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/mobilenet_7.h5',
    # 'enet_b0_8_va_mtl.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/24c7b22228f88429bf11c64f1f3f292b8e8abe32/models/affectnet_emotions/enet_b0_8_va_mtl.pt',
}

def download_model(model_url, model_name):
    import urllib.request

    if not os.path.isfile(model_name):
        urllib.request.urlretrieve(model_url, model_name)

# Prepare custom dataset

In [12]:
DISTRACTED_DIR = os.path.join(USERS_DATASET, "distracted/")
ENGAGED_DIR = os.path.join(USERS_DATASET, "engaged/")
DISTRACTED_DIR_FACES = os.path.join(USERS_DATASET, "distracted_faces/")
ENGAGED_DIR_FACES = os.path.join(USERS_DATASET, "engaged_faces/")

In [13]:
def get_total_frames(video_file):
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    return int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

def get_video_frames(video_file, n_frames=1, frame_processor=None):
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))//n_frames
    for frame_count in range(n_frames,n_frames*(total_frames+1),n_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES,frame_count)
        ret, frame_bgr = cap.read()
        if not ret:
            cap.release()
            break
        frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        if frame_processor is not None:
            frame = frame_processor(frame)
        yield frame

def preprocess(frame):
    from functools import wraps
    import sys
    import io
    def capture_output(func):
        """Wrapper to capture print output."""

        @wraps(func)
        def wrapper(*args, **kwargs):
            old_stdout = sys.stdout
            new_stdout = io.StringIO()
            sys.stdout = new_stdout
            try:
                return func(*args, **kwargs)
            finally:
                sys.stdout = old_stdout

        return wrapper
    
    detector = MTCNN()
    w_detect_face = capture_output(detector.detect_faces)
    detected = w_detect_face(frame)
    if len(detected) == 0:
        return None
    face = detected[0]
    x1, y1, w, h = face['box'][0:4]
    x2 = x1 + w
    y2 = y1 + h
    frame = frame[y1:y2, x1:x2, :]
    return cv2.resize(frame, INPUT_SIZE)

def process_videos(video_dir, result_dir):
    for vid in os.listdir(video_dir):
        vid_path = os.path.join(video_dir, vid)
        out_path = os.path.join(result_dir, vid.split('.')[0])
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        else:
            print("Skipping: ", vid)
            continue
        total_frames = get_total_frames(vid_path)
        frames = get_video_frames(vid_path, frame_processor=preprocess)
        for i, f in enumerate(frames):
            print("[{}: {}/{}] ".format(vid, i, total_frames), end='\r')
            if f is None:
                continue
            cv2.imwrite(os.path.join(out_path, str(i) + '.png'), cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        print()

In [20]:
process_videos(DISTRACTED_DIR, DISTRACTED_DIR_FACES)
process_videos(ENGAGED_DIR, ENGAGED_DIR_FACES)

FileNotFoundError: [Errno 2] No such file or directory: '/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/distracted/'

# Extract all features

In [14]:
path = '/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Egor_engaged_10/'
print(len(os.listdir(path)))

145


In [15]:
def get_frames_paths(directory):
    video2frames = {}
    for video_dir in os.listdir(directory):
        print("Dir: ", video_dir)
        path = os.path.join(directory, video_dir)
        total_files = len(os.listdir(path))
        video2frames[video_dir] = []
        i = 0
        while i < total_files:
            file_name = str(i) + ".png"
            file_path = os.path.join(path, file_name)
            i += 1
            if not os.path.exists(file_path):
                print(file_path)
                continue
            video2frames[video_dir].append(file_path)
    return video2frames

engaged2frames = get_frames_paths(ENGAGED_DIR_FACES)
distracted2frames = get_frames_paths(DISTRACTED_DIR_FACES)

Dir:  Yana_engaged_2
Dir:  Katya 16 engaged
Dir:  Den_engaged_5
Dir:  Egor_engaged_4
Dir:  Katya 1 engaged
Dir:  Egor_engaged_13
Dir:  Egor_engaged_3
Dir:  Yana_engaged_10
Dir:  Den_engaged_10
Dir:  Kate_engaged_9
Dir:  Den_engaged_7
Dir:  Katya 12 engaged
Dir:  Kate_engaged_6
Dir:  Yana_engaged_5
Dir:  Yana_engaged_6
Dir:  Den_engaged_2
Dir:  Kate_engaged_16
Dir:  Kate_engaged_4
Dir:  Katya 14 engaged
Dir:  Egor_engaged_16
Dir:  Den_engaged_4
Dir:  Den_engaged_14
Dir:  Katya 9 engaged
Dir:  Kate_engaged_7
Dir:  Katya 10 engaged
Dir:  Yana_engaged_14
Dir:  Kate_engaged_1
/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Kate_engaged_1/0.png
/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Kate_engaged_1/1.png
/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Kate_engaged_1/2.png
/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Kate_engaged_1/3.png
Dir:  Yana_engaged_7
Dir:  Yana_engaged_16
Dir:  Egor_

In [16]:
for base_model_key in model_urls.keys():
    print("Model: ", base_model_key)
    download_model(model_urls[base_model_key], base_model_key)
    base_model_name = './' + base_model_key
    torch_model = True if base_model_name[base_model_name.rfind('.') + 1:] == 'pt' else False
    if torch_model:
        BATCH_SIZE = 64
    else:
        BATCH_SIZE = 128
        
    if base_model_key in ('enet_b2_7.pt', 'enet_b2_8.pt'):
        cnn_model = torch.load(base_model_name)
        cnn_model = cnn_model.to(device)
        cnn_model.eval()

        #del cnn_model.classifier
        last_layer=torch.nn.Sequential(cnn_model.classifier,torch.nn.Softmax(dim=0))
        if True:
            inp = torch.randn(1408).to(device)
            f=last_layer.forward(inp)
            #print(f.shape,f,f.sum(axis=1))
        cnn_model.classifier=torch.nn.Identity()
        #print(cnn_model)
    elif base_model_key in ('enet_b0_8_va_mtl.pt'):
        cnn_model = torch.load(base_model_name)
        cnn_model = cnn_model.to(device)
        cnn_model.eval()

        #del cnn_model.classifier
        last_layer=torch.nn.Sequential(cnn_model.classifier,torch.nn.Softmax(dim=1))
        if True:
            inp = torch.randn(20, 1280).to(device)
            f=last_layer.forward(inp)
            #print(f.shape,f,f.sum(axis=1))
        cnn_model.classifier=torch.nn.Identity()
        #print(cnn_model)
    elif torch_model:
        cnn_model = torch.load(base_model_name)
        cnn_model = cnn_model.to(device)
        cnn_model.eval()

        #del cnn_model.classifier
        last_layer=torch.nn.Sequential(cnn_model.classifier[0],torch.nn.Softmax(dim=1))
        if True:
            inp = torch.randn(20, 1280).to(device)
            f=last_layer.forward(inp)
            #print(f.shape,f,f.sum(axis=1))
        cnn_model.classifier=torch.nn.Identity()
    else:
        base_model = load_model(base_model_name)
        layer_out = base_model.get_layer('global_pooling')
        cnn_model = Model(base_model.input,layer_out.output)

        #cnn_model.summary()
    
    engaged_pickle = FEATURES_DIR + 'engaged_features_{}.pickle'.format(base_model_key)
    distracted_pickle = FEATURES_DIR + 'distracted_features_{}.pickle'.format(base_model_key)
    print("Engaged:")
    _ = extract_or_read_features(engaged_pickle, engaged2frames)
    
    print("Distracted:")
    _ = extract_or_read_features(distracted_pickle, distracted2frames)

Model:  enet_b0_8_best_afew.pt
Engaged:
Distracted:
Model:  mobilenet_7.h5


2024-08-11 09:01:49.918716: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 32621 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Engaged:


2024-08-11 09:01:51.477863: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902


Distracted:


# Read features

In [17]:
base_model_key = 'enet_b0_8_best_afew.pt'

In [18]:
engaged_pickle = FEATURES_DIR + 'engaged_features_{}.pickle'.format(base_model_key)
eng_vid2features = load_features(engaged_pickle)

In [19]:
print(eng_vid2features['Yana_engaged_2'])

[[-0.17457296 -0.0838912   0.0130755  ... -0.13524437 -0.19775935
   0.08375152]
 [-0.17530565 -0.09148189 -0.00201505 ... -0.1188405  -0.20162667
   0.13017893]
 [-0.17212895 -0.09340156 -0.00343952 ... -0.1270643  -0.2020545
   0.14086863]
 ...
 [-0.15957235 -0.10684593 -0.04282501 ... -0.12603702 -0.05989177
   0.14404123]
 [-0.12564267 -0.12438735 -0.04556637 ... -0.1355438  -0.12425207
   0.16872056]
 [-0.09237796 -0.11252303 -0.07996154 ... -0.17252916 -0.15970832
   0.07588568]]
